In [ ]:
import re
from collections import Counter

import numpy as np

In [ ]:
class Shape:
    # Present shape
    def __init__(self, rows):
        assert(len(rows) == 3)
        assert(all(len(row) == 3 for row in rows))
        self.shape = np.array(rows, dtype=bool)
        self.width = self.shape.shape[1]
        self.length = self.shape.shape[0]

    def __str__(self):
        return f'Dimensions: {self.width}x{self.length}. Area: {self.area}.'

    @property
    def area(self):
        return self.shape.sum()

In [ ]:
class Region:
    def __init__(self, width, length, shape_quantities):
        self.width = width
        self.length = length
        self.shape_quantities = shape_quantities

    def __str__(self):
        return f'Dimensions: {self.width}x{self.length}. Area: {self.area}. Presents area: {self.presents_area}. Presents: {self.presents_total}. Presents easily accommodated: {self.present_trivially_fitting}.'

    @property
    def trivial_fit(self):
        return self.present_trivially_fitting >= self.presents_total

    @property
    def present_trivially_fitting(self):
        return (self.width // self.shape_max_dimension) * (self.length // self.shape_max_dimension)

    @property
    def shape_max_dimension(self):
        return max(max(shape.width, shape.length) for shape in self.shape_quantities.keys())

    @property
    def presents_total(self):
        return self.shape_quantities.total()

    @property
    def trivial_unfit(self):
        return self.presents_area > self.area

    @property
    def presents_area(self):
        return sum(shape.area * count for shape, count in self.shape_quantities.items())

    @property
    def area(self):
        return self.width * self.length

In [ ]:
pattern_shape_index = re.compile(r'(?P<index>\d+):')
pattern_shape_row = re.compile(r'(?P<row>[.#]+)')
pattern_region = re.compile(r'(?P<width>\d+)x(?P<length>\d+): (?P<shape_quantities>[\d ]+)')

def read_instance(f):
    shapes = {}
    regions = []
    shape_index = None
    shape_rows = None
    for line in f:
        line = line.strip()
        if line == '':
            if shape_rows is not None:
                assert(shape_index is not None)
                assert(shape_index not in shapes)
                shapes[shape_index] = Shape(shape_rows)
                shape_index = None
                shape_rows = None
            continue
        m = pattern_shape_index.fullmatch(line)
        if m:
            assert(shape_index is None)
            shape_index = int(m['index'])
            shape_rows = []
            continue
        m = pattern_shape_row.fullmatch(line)
        if m:
            shape_row = m['row']
            assert(all(c in {'.', '#'} for c in shape_row))
            shape_rows.append([c == '#' for c in shape_row])
            continue
        m = pattern_region.fullmatch(line)
        if m:
            shape_quantities = list(map(int, m['shape_quantities'].split()))
            shapes_counter = Counter({shapes[i]: q for i, q in enumerate(shape_quantities)})
            regions.append(Region(int(m['width']), int(m['length']), shapes_counter))
    return shapes, regions

In [ ]:
def solve_file(f):
    shapes, regions = read_instance(f)
    regions_fit = 0
    regions_unfit = 0
    for region in regions:
        assert not (region.trivial_unfit and region.trivial_fit)
        if region.trivial_fit:
            regions_fit += 1
        if region.trivial_unfit:
            regions_unfit += 1
    return regions_fit, len(regions) - regions_unfit

In [ ]:
with open('input.txt') as f:
    print(solve_file(f)) # 476

In [ ]:
with open('example.txt') as f:
    print(solve_file(f))